# **IMPORTAR BIBLIOTECAS NECESARIAS Y CARGAR DATOS**

In [13]:
# BIBLIOTECAS NECESARIAS

# Bibliotecas para operaciones numéricas y manejo de datos
import numpy as np  # Operaciones numéricas
import pandas as pd  # Manejo de datos en formato tabular

# División del conjunto de datos y validación cruzada
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

# Modelos de clasificación
import xgboost as xgb

# Importar todo el módulo de métricas
import sklearn.metrics

# Métricas de evaluación y reportes
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    PrecisionRecallDisplay,
    roc_curve, auc,
    classification_report,
    accuracy_score, precision_score, recall_score, f1_score
)

# Bibliotecas para gráficos
import matplotlib.pyplot as plt  # Gráficos básicos
import seaborn as sns  # Gráficos estadísticos, con menos código que matplotlib
import plotly.express as px  # Gráficos interactivos con interfaz sencilla y de alto nivel
import plotly.graph_objects as go  # Gráficos interactivos con interfaz más detallada y de bajo nivel

# Herramientas para iteradores
import itertools  # Herramientas funcionales para crear y utilizar iteradores

# Herramientas para preprocesamiento de datos y evaluación de modelos
from sklearn.preprocessing import StandardScaler  # Escalado de características para normalizar datos
from sklearn.model_selection import learning_curve  # Generación de curvas de aprendizaje para evaluar el rendimiento del modelo
from sklearn.model_selection import validation_curve  # Generación de curvas de validación para evaluar el rendimiento del modelo

In [5]:
# Cargar el dataset desde un archivo CSV
credit_card = pd.read_csv('C:\\Users\\mmene\\OneDrive\\Escritorio\\TESIS\\BD2\\default_of_credit_card_clients.csv', sep=';')

In [6]:
credit_card.head(10)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
5,50000,1,1,2,37,0,0,0,0,0,...,19394,19619,20024,2500,1815,657,1000,1000,800,0
6,500000,1,1,2,29,0,0,0,0,0,...,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
7,100000,2,2,2,23,0,-1,-1,0,0,...,221,-159,567,380,601,0,581,1687,1542,0
8,140000,2,3,1,28,0,0,2,0,0,...,12211,11793,3719,3329,0,432,1000,1000,1000,0
9,20000,1,3,2,35,-2,-2,-2,-2,-1,...,0,13007,13912,0,0,0,13007,1122,0,0


# **NORMALIZACIÓN**

In [8]:
# Paso 1: Separar características (X) y la columna objetivo (y)
X = credit_card.drop(['Y'], axis=1)  # Eliminar la columna 'Y'
y = credit_card['Y']  # Conservar la columna 'Y'

# Paso 2: Crear un objeto StandardScaler
scaler = StandardScaler()

# Paso 3: Ajustar el escalador a las características y transformarlas
scaled = scaler.fit_transform(X)

# Paso 4: Convertir X_scaled de vuelta a un DataFrame
X_scaled = pd.DataFrame(scaled, columns=X.columns)

# Paso 5: Reunir X_scaled con la columna 'Class'
credit_card_scaled = pd.concat([X_scaled, y.reset_index(drop=True)], axis=1)

# Paso 6: Mostrar las primeras filas del DataFrame transformado
credit_card_scaled.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,-1.136720,0.810161,0.185828,-1.057295,-1.246020,1.794564,1.782348,-0.696663,-0.666599,-1.530046,...,-0.672497,-0.663059,-0.652724,-0.341942,-0.227086,-0.296801,-0.308063,-0.314136,-0.293382,1
1,-0.365981,0.810161,0.185828,0.858557,-1.029047,-0.874991,1.782348,0.138865,0.188746,0.234917,...,-0.621636,-0.606229,-0.597966,-0.341942,-0.213588,-0.240005,-0.244230,-0.314136,-0.180878,1
2,-0.597202,0.810161,0.185828,0.858557,-0.161156,0.014861,0.111736,0.138865,0.188746,0.234917,...,-0.449730,-0.417188,-0.391630,-0.250292,-0.191887,-0.240005,-0.244230,-0.248683,-0.012122,0
3,-0.905498,0.810161,0.185828,-1.057295,0.164303,0.014861,0.111736,0.138865,0.188746,0.234917,...,-0.232373,-0.186729,-0.156579,-0.221191,-0.169361,-0.228645,-0.237846,-0.244166,-0.237130,0
4,-0.905498,-1.234323,0.185828,-1.057295,2.334029,-0.874991,0.111736,-0.696663,0.188746,0.234917,...,-0.346997,-0.348137,-0.331482,-0.221191,1.335034,0.271165,0.266434,-0.269039,-0.255187,0


In [11]:
credit_card_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      30000 non-null  float64
 1   X2      30000 non-null  float64
 2   X3      30000 non-null  float64
 3   X4      30000 non-null  float64
 4   X5      30000 non-null  float64
 5   X6      30000 non-null  float64
 6   X7      30000 non-null  float64
 7   X8      30000 non-null  float64
 8   X9      30000 non-null  float64
 9   X10     30000 non-null  float64
 10  X11     30000 non-null  float64
 11  X12     30000 non-null  float64
 12  X13     30000 non-null  float64
 13  X14     30000 non-null  float64
 14  X15     30000 non-null  float64
 15  X16     30000 non-null  float64
 16  X17     30000 non-null  float64
 17  X18     30000 non-null  float64
 18  X19     30000 non-null  float64
 19  X20     30000 non-null  float64
 20  X21     30000 non-null  float64
 21  X22     30000 non-null  float64
 22

In [12]:
# VALORES ÚNICOS DE LA COLUMNA 'Class'
# 1. Crea un conjunto de los valores únicos presentes en la columna 'Class' del DataFrame
# 2. Muestra los valores únicos

value_etq = set(credit_card_scaled['Y'])
print("\033[1mVARIABLE RESPUESTA:\033[0m")
print("\nToma el valor de \033[1m1\033[0m en caso de \033[1mIMPAGO\033[0m y toma el valor de \033[1m0\033[0m (SI PAGÓ)\nen \033[1mcaso contrario\033[0m\n")
print(f'Clases: {value_etq}')

credit_card_scaled['Y'].value_counts()

VARIABLE RESPUESTA:

Toma el valor de 1 en caso de IMPAGO y toma el valor de 0 (SI PAGÓ)
en caso contrario

Clases: {0, 1}


Y
0    23364
1     6636
Name: count, dtype: int64

# **MODELO XGBoost**

In [23]:
# 'HOLDOUT METHOD' para evaluar el rendimiento de un modelo de aprendizaje automático
# Este método divide el conjunto de datos en entrenamiento y prueba con diferentes tamaños de entrenamiento
# para evaluar cómo el rendimiento del modelo varía con el tamaño del conjunto de entrenamiento.

# Definir los diferentes tamaños para el conjunto de entrenamiento
train_sizes = [0.75,0.80,0.90]

# Crear una lista para almacenar los resultados de los diferentes conjuntos de entrenamiento y prueba
split_results = []

# Para cada tamaño de entrenamiento en train_sizes
for splits in train_sizes:
  # Se divide los datos X e y en conjuntos de entrenamiento y prueba con el tamaño especificado
  X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, train_size=splits, random_state=42)
  split_results.append((X_train, X_test, y_train, y_test))

# Mostrar las dimensiones de los conjuntos de entrenamiento y prueba para cada tamaño de entrenamiento
for i, (X_train, X_test, y_train, y_test) in enumerate(split_results):
  print(f"\033[1mConjunto con train_size={train_sizes[i]}\033[0m")
  print(f"Dimensión total es: {credit_card_scaled.shape}")
  print(f"X_train dimensión: {X_train.shape}, X_test dimensión: {X_test.shape}")
  print(f"y_train dimensión: {y_train.shape}, y_test dimensión: {y_test.shape}")
  print()

Conjunto con train_size=0.75
Dimensión total es: (30000, 24)
X_train dimensión: (22500, 23), X_test dimensión: (7500, 23)
y_train dimensión: (22500,), y_test dimensión: (7500,)

Conjunto con train_size=0.8
Dimensión total es: (30000, 24)
X_train dimensión: (24000, 23), X_test dimensión: (6000, 23)
y_train dimensión: (24000,), y_test dimensión: (6000,)

Conjunto con train_size=0.9
Dimensión total es: (30000, 24)
X_train dimensión: (27000, 23), X_test dimensión: (3000, 23)
y_train dimensión: (27000,), y_test dimensión: (3000,)



In [27]:
# MODELO XGBoost

# Lista para almacenar el resultado del modelo para cada conjunto de entrenamiento/prueba
models_split_results = []

# Iterar sobre los conjuntos generados
for i, (X_train, X_test, y_train, y_test) in enumerate(split_results):

  # Definir las métricas de evaluación
  scores = ["precision", "recall", "roc_auc", "f1", "average_precision"]

  # Mostrar información sobre el conjunto actual
  print(f"\033[1mConjunto con train_size={train_sizes[i]}:\033[0m")

  # Crear el modelo XGBoost

  modelo_xgb = xgb.XGBClassifier(random_state=42, objective='binary:logistic')

  # Definir los parámetros para GridSearchCV
  grid_search = {
      "n_estimators": [10,20,50,100,200,500], # Número de árboles en el bosque aleatorio
      "max_depth": [3,4,7,9], # Profundidad máxima de cada
      "learning_rate": [0.02,0.15,0.2,0.4,0.5],
      "gamma": [0,0.25,0.5],
      "lambda": [0,0.25,0.5],
      "alpha": [0,0.25,0.5],
      "booster": ["gbtree"]
      }

  # Medir tiempo de ejecución
  start = pd.Timestamp.now()

  # Configuración de GridSearchCV
  grid = GridSearchCV(
      modelo_xgb,
      grid_search,
      cv=3, # Número de particiones en la validación cruzada
      refit=False, # No volver a entrenar el mejor modelo encontrado en toda la búsqueda
      scoring=scores, # Métricas de evaluación a usar
      verbose = 3, # Mostrar información adicional durante la búsqueda
      return_train_score=True # Devolver los resultados de entrenamiento
      )

  # Ejecutar GridSearchCV para encontrar el mejor modelo
  model_grid = grid.fit(X_train, y_train)

  # Medir tiempo de ejecución de GridSearchCV
  end = pd.Timestamp.now()
  print(f"Duración búsqueda: {end-start}")

  # Almacenar los resultados en la list 'models_split_results'
  models_split_results.append(model_grid.cv_results_)

  # Guardar modelos en un DataFrame
modelos_GSCV1= pd.DataFrame(models_split_results)

model_75 = models_split_results[0]
model_80 = models_split_results[1]
model_90 = models_split_results[2]

#75
# Crear una lista de diccionarios, donde cada diccionario corresponde a una ejecución
ajuste_modelo_75 = []

# Recorre cada índice 'i' basado en la longitud de los tiempos medios de ajuste ('mean_fit_time')
for i in range(len(model_75['mean_fit_time'])):
  # Crea un diccionario 'execution' para la ejecución actual, extrayendo el valor correspondiente al índice 'i' de cada clave en 'resultados_modelo_75'
  execution = {key: value[i] for key, value in model_75.items()}
  # Añade el diccionario de resultados de la ejecución actual a la lista 'ajuste_modelo_80'
  ajuste_modelo_75.append(execution)

# Convierte la lista de diccionarios 'ajuste_modelo_75' en un DataFrame de pandas para facilitar su manipulación y análisis
modelos_75 = pd.DataFrame(ajuste_modelo_75)
# Guarda el DataFrame resultante en un archivo Excel, permitiendo almacenar los resultados de manera estructurada
modelos_75.to_excel('C:\\Users\\mmene\\OneDrive\\Escritorio\\TESIS\\XGBoost\\PRUEBA_RESULTADO\\modelos_75.xlsx')

#80
# Crear una lista de diccionarios, donde cada diccionario corresponde a una ejecución
ajuste_modelo_80 = []

# Recorre cada índice 'i' basado en la longitud de los tiempos medios de ajuste ('mean_fit_time')
for i in range(len(model_80['mean_fit_time'])):
  # Crea un diccionario 'execution' para la ejecución actual, extrayendo el valor correspondiente al índice 'i' de cada clave en 'resultados_modelo_75'
  execution = {key: value[i] for key, value in model_80.items()}
  # Añade el diccionario de resultados de la ejecución actual a la lista 'ajuste_modelo_80'
  ajuste_modelo_80.append(execution)

# Convierte la lista de diccionarios 'ajuste_modelo_80' en un DataFrame de pandas para facilitar su manipulación y análisis
modelos_80 = pd.DataFrame(ajuste_modelo_80)
# Guarda el DataFrame resultante en un archivo Excel, permitiendo almacenar los resultados de manera estructurada
modelos_80.to_excel('C:\\Users\\mmene\\OneDrive\\Escritorio\\TESIS\\XGBoost\\PRUEBA_RESULTADO\\modelos_80.xlsx')


#90
# Crear una lista de diccionarios, donde cada diccionario corresponde a una ejecución
ajuste_modelo_90 = []

# Recorre cada índice 'i' basado en la longitud de los tiempos medios de ajuste ('mean_fit_time')
for i in range(len(model_90['mean_fit_time'])):
  # Crea un diccionario 'execution' para la ejecución actual, extrayendo el valor correspondiente al índice 'i' de cada clave en 'resultados_modelo_75'
  execution = {key: value[i] for key, value in model_90.items()}
  # Añade el diccionario de resultados de la ejecución actual a la lista 'ajuste_modelo_80'
  ajuste_modelo_90.append(execution)

# Convierte la lista de diccionarios 'ajuste_modelo_90' en un DataFrame de pandas para facilitar su manipulación y análisis
modelos_90 = pd.DataFrame(ajuste_modelo_90)
# Guarda el DataFrame resultante en un archivo Excel, permitiendo almacenar los resultados de manera estructurada
modelos_90.to_excel('C:\\Users\\mmene\\OneDrive\\Escritorio\\TESIS\\XGBoost\\PRUEBA_RESULTADO\\modelos_90.xlsx')

Conjunto con train_size=0.75:
Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV 1/3] END alpha=0, booster=gbtree, gamma=0, lambda=0, learning_rate=0.02, max_depth=7, n_estimators=100; average_precision: (train=0.703, test=0.570) f1: (train=0.551, test=0.471) precision: (train=0.807, test=0.700) recall: (train=0.419, test=0.355) roc_auc: (train=0.858, test=0.782) total time=   0.3s
[CV 2/3] END alpha=0, booster=gbtree, gamma=0, lambda=0, learning_rate=0.02, max_depth=7, n_estimators=100; average_precision: (train=0.715, test=0.549) f1: (train=0.546, test=0.430) precision: (train=0.836, test=0.689) recall: (train=0.406, test=0.313) roc_auc: (train=0.864, test=0.778) total time=   0.3s
[CV 3/3] END alpha=0, booster=gbtree, gamma=0, lambda=0, learning_rate=0.02, max_depth=7, n_estimators=100; average_precision: (train=0.712, test=0.537) f1: (train=0.552, test=0.457) precision: (train=0.808, test=0.664) recall: (train=0.419, test=0.348) roc_auc: (train=0.858, test=0.769) tot

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\mmene\\OneDrive\\Escritorio\\TESIS\\XGBoost\\PRUEBA_RESULTADO\\modelos_75.xlsx'